In [1]:
from Acesso import Login
from Query import Query
import pandas as pd
import requests

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

links='https://api.powerbi.com/beta/89024f02-ca0e-4816-b5b2-1b5bdf790280/datasets/220441ac-f887-4ef5-9ddb-4d84e0b5c7a0/rows?noSignUpCheck=1&key=hD9mYpvCj%2FbmyoM1biuGj8KpmOXOcbBBumogzXvTUQR9EPmx9rXA5CjBzu7BsBrpnNro0tqFRZzTku1Z3ux6OQ%3D%3D'

pd.set_option('display.max_columns',None)

querys={

    'TargetEstatico':

    """
    
    DECLARE @DTBASE DATETIME,@DTFIM DATETIME,@DTINICIO DATETIME

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)

    SET @DTFIM=@DTBASE

    SET @DTINICIO=CONCAT(YEAR(@DTFIM),'-',MONTH(@DTFIM),'-01')

    SELECT * FROM netfeira.vw_targetestatico
    WHERE [Data de Faturamento] BETWEEN @DTINICIO AND @DTFIM
    ORDER BY [Data de Faturamento]

    
    """,
    
    'Aberto':
    
    """
    
    SELECT * FROM netfeira.vw_aberto
    WHERE [Data do Pedido]=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)
    
    """,
    
    'Meta':
    
    """
    
    SELECT * FROM netfeira.vw_metas
    
    """,
    
    'Calendario':
    
    
    """
    
    SELECT * FROM netfeira.vw_calend
    WHERE YEAR(Data)=YEAR(GETDATE()) AND MONTH(Data)=MONTH(GETDATE())
    
    """,
    
    'Vendedor':
    
    """
    
    SELECT * FROM netfeira.vw_vendedor
    
    """,
    
    
    'Supervisor':
    
    
    """
    
    SELECT * FROM netfeira.vw_supervisor
    
    """,
    
    'Produto':
    
    """
    
    SELECT * FROM netfeira.vw_produto
    
    """,
    
    'Carteira':
    
    """
    
    SELECT * FROM netfeira.vw_carteira
    WHERE Dias<366 AND [Status do Cliente]='ATIVO' 
    
    """,
    
    'MIX':
    
    """
    
    SELECT * FROM netfeira.vw_mix
    
    """

}

In [2]:
tabelas_df=sql.CriarTabela(kwargs=querys)

In [3]:
supervisor_df=pd.DataFrame()

supervisor_df=tabelas_df['Supervisor'].loc[tabelas_df['Supervisor']['Equipe'].str.contains('9')]

In [4]:
vendedor_df=pd.DataFrame()

vendedor_df=tabelas_df['Vendedor'].merge(supervisor_df,on='ID Equipe',how='inner')

In [5]:
vendedor_df=vendedor_df.loc[vendedor_df['Status do Vendedor']=='ATIVO']

In [6]:
meta_df=pd.DataFrame()

meta_df=vendedor_df.merge(tabelas_df['Meta'],on='ID Vendedor',how='inner')

meta_df=meta_df.loc[meta_df['Meta R$']>0]

meta_df.sort_values('Meta R$',ascending=False,inplace=True)

In [7]:
meta_df

,ID Vendedor,Vendedor,Nome Resumido,ID Equipe,E-mail,Categoria,Data de Cadastro,Status do Vendedor,Equipe,ID Sup,Supervisor,Email Sup,ID Gerente,Gerente,Email Gerente,Meta R$
5,RENATAAP,RENATA APARECIDA NOCHIERI COSTA,RENATA COSTA,0009,RENATA@DEMARCHISAOPAULO.COM.BR,CLT,2021-01-29,ATIVO,EQUIPE 9 (ATIVO),RAQUELSO,RAQUEL SOBRINHO,RAQUEL@DEMARCHISAOPAULO.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,182000.0
4,RAQUELSO,RAQUEL SOBRINHO,RAQUEL SOBRINHO,0009,RAQUEL@DEMARCHISAOPAULO.COM.BR,CLT,2021-01-29,ATIVO,EQUIPE 9 (ATIVO),RAQUELSO,RAQUEL SOBRINHO,RAQUEL@DEMARCHISAOPAULO.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,169817.5
3,MAYRAQS,MAYRA QUEIROS DOS SANTOS,MAYRA SANTOS,0009,MAYRA.SANTOS@DEMARCHISAOPAULO.COM.BR,CLT,2021-09-13,ATIVO,EQUIPE 9 (ATIVO),RAQUELSO,RAQUEL SOBRINHO,RAQUEL@DEMARCHISAOPAULO.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,110000.0
6,TALITA,TALITA ALVES DE OLIVEIRA,TALITA OLIVEIRA,0009,TALITA.OLIVEIRA@DEMARCHISAOPAULO.COM.BR,CLT,2022-01-26,ATIVO,EQUIPE 9 (ATIVO),RAQUELSO,RAQUEL SOBRINHO,RAQUEL@DEMARCHISAOPAULO.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,110000.0
2,JUVILARE,JULIANA COSTA VILARES DOS SANTOS,JULIANA SANTOS,0009,JULIANA.COSTA@DEMARCHISAOPAULO.COM.BR,CLT,2022-08-29,ATIVO,EQUIPE 9 (ATIVO),RAQUELSO,RAQUEL SOBRINHO,RAQUEL@DEMARCHISAOPAULO.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,25000.0
7,THAISFRE,THAIS SANTOS DE FREITAS,THAIS FREITAS,0009,THAIS.SANTOS@DEMARCHISAOPAULO.COM.BR,CLT,2022-08-25,ATIVO,EQUIPE 9 (ATIVO),RAQUELSO,RAQUEL SOBRINHO,RAQUEL@DEMARCHISAOPAULO.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,25000.0


In [8]:
calend_df=pd.DataFrame()

calend_df=tabelas_df['Calendario']

uteis=len(calend_df['Data'].loc[calend_df['Dia Útil']==True].unique().tolist())

trabalhado=len(calend_df['Data Trabalhada'].loc[(calend_df['Dia Útil']==True)&(~calend_df['Data Trabalhada'].isnull())].unique().tolist())-1

restante=uteis-trabalhado

In [9]:
uteis

21

In [10]:
for indice,linha in meta_df.iterrows():
    
    vendas_df=pd.DataFrame()
    
    vendas_df=tabelas_df['TargetEstatico'].loc[(tabelas_df['TargetEstatico']['ID Vendedor']==linha['ID Vendedor'])&(tabelas_df['TargetEstatico']['Tipo de Operação']=='VENDAS')]
    
    aberto_df=pd.DataFrame()
    
    aberto_df=tabelas_df['Aberto'].loc[tabelas_df['Aberto']['ID Vendedor']==linha['ID Vendedor']]
    
    produto_df=pd.DataFrame()

    produto_df=tabelas_df['MIX'].loc[tabelas_df['MIX']['ID Vendedor']==linha['ID Vendedor']]
    
    situacao_df=pd.DataFrame()
    
    situacao_df=vendas_df[['Situação','Total Venda']].groupby(['Situação'],as_index=False).sum()
        
    situacao_df['Cliente']=situacao_df['Situação'].apply(lambda info: len(vendas_df['ID Cliente'].loc[vendas_df['Situação']==info].unique().tolist()))
    
    situacao_df['Pedido']=situacao_df['Situação'].apply(lambda info: len(vendas_df['Pedido'].loc[vendas_df['Situação']==info].unique().tolist()))
    
    situacao_df['MIX']=situacao_df['Situação'].apply(lambda info: len(vendas_df['SKU'].loc[vendas_df['Situação']==info].unique().tolist()))

    situacao_df.loc[situacao_df['Situação']!='FATURADO','Cliente']=situacao_df['Cliente'].loc[situacao_df['Situação']!='FATURADO']*-1
    
    situacao_df.loc[situacao_df['Situação']!='FATURADO','Pedido']=situacao_df['Pedido'].loc[situacao_df['Situação']!='FATURADO']*-1
    
    situacao_df.loc[situacao_df['Situação']!='FATURADO','MIX']=situacao_df['MIX'].loc[situacao_df['Situação']!='FATURADO']*-1
    
    cliente=situacao_df['Cliente'].sum()
            
    pedido=situacao_df['Pedido'].sum()
    
    mix=situacao_df['MIX'].sum()
    
    meta=linha['Meta R$']
    
    faturado=vendas_df['Total Venda'].sum()
    
    perc_meta=round(faturado/meta,4)*100
    
    carteira=len(tabelas_df['Carteira']['ID Cliente'].loc[(tabelas_df['Carteira']['ID Vendedor']==linha['ID Vendedor'])].unique().tolist())
    
    qtd_cli=cliente if cliente>0 else 0
    
    perc_carteira=round(qtd_cli/carteira,4)*100
    
    dif_cart=carteira-cliente
    
    minimo=0
    
    maximo=100
    
    ticket=round(faturado/pedido,2)
    
    dif_meta=round(faturado-meta,2)
    
    projecao=round((faturado/trabalhado)*uteis,2)
    
    #pedidos em aberto atual
    
    atendimento=len(aberto_df['ID Cliente'].unique().tolist())
    
    pedido_real=len(aberto_df['Pedido'].unique().tolist())
    
    realizado=aberto_df['Total Venda'].sum()
    
    meta_diaria=round(meta/uteis,2)
    
    em_aberto=aberto_df['Total Venda'].loc[aberto_df['Situação']=='AB'].sum()
    
    total=round(faturado+em_aberto,2)
    
    perc_diario=round(realizado/meta_diaria,4)*100
    
    dif_diario=round(realizado-meta_diaria,2)
    
    kg_real=aberto_df['Peso Bruto KG'].sum()
    
    break
    
    pass

In [11]:
produto_df.columns

Index(['ID Vendedor', 'SKU', 'Seq', 'Positivado', 'Qtde'], dtype='object')

In [12]:
produto_df=produto_df.merge(tabelas_df['Produto'],on='SKU',how='inner')[['SKU','Produto', 'Seq', 'Positivado', 'Qtde']]

In [15]:
produto_df.loc[produto_df['Positivado']==0,'Qtde']=0

In [16]:
produto_df.loc[produto_df['Positivado']==1].sort_values('Qtde',ascending=False)

,SKU,Produto,Seq,Positivado,Qtde
27,9493,PÃO HB PN10 BRIOCHE ARYZTA FP (PCT 4UNID X 64G),244,1,156.0
37,9492,PÃO HB PN10 AUSTRALIANO FP (PCT 4UNID X 64G),243,1,113.0
31,442,IQF ABACAXI (100G),36,1,100.0
15,1080,"IQF FRUTAS VERMELHAS CONGELADA 1,200 KG",145,1,100.0
14,440,"IQF MORANGO (1,2 KG)",35,1,96.0
...,...,...,...,...,...
59,1113,CONG. BATATA MCCAIN 9MM TRAD 720G (12X720GR),155,1,1.0
33,1003,"CONGELADO LEGUMES PARA SALTEAR 1,2KG",112,1,1.0
35,1022,CONGELADO PAO DE QUEIJO 25GR TRAD (1KG) DE MARCHI,120,1,1.0
41,730,CONGELADO MANDIOCA PALITO DM 1.2 KG,79,1,1.0


In [17]:
produto_df.loc[produto_df['Positivado']==0].sort_values('Seq',ascending=True)

,SKU,Produto,Seq,Positivado,Qtde
255,74,POLPINHA DE COCO (100G),6,0,0.0
86,96,POLPINHA DE FRAMBOESA (100G),9,0,0.0
77,176,POLPINHA DE PESSEGO (100G),14,0,0.0
158,216,POLPINHA DE TAMARINDO 100G),15,0,0.0
145,347,IQF AMORA CONGELADA POTE (450G),19,0,0.0
...,...,...,...,...,...
149,9567,SUCO DE UVA 300ML DEMARCHI,262,0,0.0
253,9569,SUCO DE ABACAXI HORT 300ML DEMARCHI,263,0,0.0
215,9584,CONGELADO COUVE FLOR ATIGEL (1.5KG),264,0,0.0
244,9586,"CONG. PICKERS CROQUETE SUINO MCCAIN 1,05KG",265,0,0.0


In [22]:
produto_df.head(10)

,SKU,Produto,Seq,Positivado,Qtde
0,345,"CONGELADO MANDIOQUINHA ( 1,2KG )",18,1,11.0
1,405,POLPINHA DE FRUTAS VERMELHAS (100G),23,1,5.0
2,9399,"CONGELADO LEGUMES PARA SALTEAR (1,020KG)",215,1,10.0
3,474,"CONGELADO CENOURA CUBOS ( 1,2KG )",44,1,5.0
4,9396,"IQF MORANGO (1,020 KG)",212,1,8.0
5,139,POLPINHA DE MARACUJA (100G),12,1,51.0
6,594,"CONGELADO BATATA MCCAIN 9MM TRADICIONAL 2,5",59,1,84.0
7,9439,"PÃO HB BRIOCHE 4"" CT (15X64G)",224,1,16.0
8,471,"CONGELADO BATATA CUBOS DM ( 1,2KG )",42,1,4.0
9,975,"CONGELADO MANDIOCA TOLETE MATHEUS (2,5)",109,1,57.0
